In [1]:
!pip install ipyaggrid pandas ipywidgets "openai>1" > /dev/null 2>&1
!echo $?

0


In [2]:
import openai
from kamiwaza.serving.services import ServingService

serving_service = ServingService()
valid_deployments = [ d for d in serving_service.list_deployments(with_names=True) if d.status != 'STOPPED' and d.instances ]

if valid_deployments:
    print(f"Found a deployment of {valid_deployments[0].m_name} - using it")
    deployment = valid_deployments[0]
else:
    print("No deployments available")

Found a deployment of Dracarys2-72B-Instruct - using it


In [3]:
openai.api_key = "local"
openai.api_base = f"http://localhost:{deployment.lb_port}/v1"
print(f"OpenAI client instantiated against Kamiwaza Deployment: {openai.api_base}")
openai.verify_ssl_certs = False

OpenAI client instantiated against Kamiwaza Deployment: http://localhost:51100/v1


In [4]:
client = openai.OpenAI(api_key='kamiwaza', base_url=f"http://localhost:{deployment.lb_port}/v1")

def infer(prompt: str, system: str = "You are a helpful assistant", temperature: float = 0.7):
    messages = [{"role": "system", "content": system}, {"role": "user", "content": prompt}]
    try:
        response = client.chat.completions.create(
            model = "model",
            messages = messages,
            temperature = temperature
        )
        return response.choices[0].message.content
    except:
        print("An error occurred:")
        traceback.print_exc()

In [5]:
print(infer("write a haiku about kamiwaza (4 syllables!), the enterprise ai stack"))

INFO:httpx:HTTP Request: POST http://localhost:51100/v1/chat/completions "HTTP/1.1 200 OK"


Kamiwaza flows,  
AI stack for business needs,  
Smart solutions rise.


In [6]:
# First, install required packages if you haven't already
# !pip install ipyaggrid pandas ipywidgets

import pandas as pd
from ipyaggrid import Grid
from ipywidgets import Button, Output, Text, VBox, HBox, Layout, HTML
from IPython.display import display, clear_output, Javascript
import json
from functools import partial

class SmartTransformGrid:
    def __init__(self, data):
        self.df = pd.DataFrame(data)
        self.setup_grid()
        self.setup_ui()
        self.setup_keyboard_shortcuts()
        
    def setup_grid(self):
        self.grid_options = {
            'columnDefs': [
                {'headerName': col, 
                 'field': col, 
                 'sortable': True, 
                 'filter': True,
                 'width': 400 if col == 'content' else 200,
                 'wrapText': True,
                 'autoHeight': True,
                 'checkboxSelection': True,
                 'headerCheckboxSelection': True
                } for col in self.df.columns
            ],
            'defaultColDef': {
                'resizable': True,
                'filterParams': {'newRowsAction': 'keep'},
                'selectable': True
            },
            'rowSelection': 'multiple',
            'suppressRowClickSelection': True,
            'domLayout': 'autoHeight',
            'enableRangeSelection': True,
            'rowMultiSelectWithClick': True,
            # Add event listener for selection changes
            'onSelectionChanged': '''
                function() {
                    const selectedRows = this.api.getSelectedRows();
                    const selectedCount = selectedRows.length;
                    document.dispatchEvent(new CustomEvent('gridSelectionChanged', {
                        detail: { count: selectedCount }
                    }));
                }
            '''
        }

        self.main_grid = Grid(
            grid_data=self.df,
            grid_options=self.grid_options,
            height=300
        )

    def setup_keyboard_shortcuts(self):
        # Add Javascript to handle Ctrl+E
        js_code = """
        document.addEventListener('keydown', function(event) {
            if (event.ctrlKey && event.key === 'e') {
                event.preventDefault();
                document.dispatchEvent(new CustomEvent('triggerTransform'));
            }
        });
        """
        display(Javascript(js_code))
        
        # Add Javascript to trigger transform
        js_trigger = """
        document.addEventListener('triggerTransform', function() {
            const transformButton = document.querySelector('button:contains("Transform Selected")');
            if (transformButton) {
                transformButton.click();
            }
        });
        """
        display(Javascript(js_trigger))

    def setup_ui(self):
        self.transform_button = Button(
            description='Transform Selected (Ctrl+E)', 
            layout=Layout(width='200px')
        )
        self.transform_all_button = Button(
            description='Transform All', 
            layout=Layout(width='150px')
        )
        self.output = Output()
        
        self.transform_button.on_click(self.handle_transform_click)
        self.transform_all_button.on_click(self.handle_transform_all_click)

    def get_selection_info(self):
        """Get information about current selection"""
        try:
            selected = self.main_grid.get_grid().get_selected_rows()
            if selected and len(selected) > 0:
                return {
                    'type': 'cells',
                    'data': selected,
                    'headers': list(self.df.columns)
                }
        except:
            # If there's any error getting selection, check if we have rows in grid
            if len(self.df) > 0:
                # Fall back to checking grid's internal state
                grid_js = """
                var grid = document.querySelector('.ag-grid-container').gridOptions.api;
                return grid.getSelectedRows().length;
                """
                display(Javascript(grid_js))
                
        return None

    def transform_data(self, selection_info):
        """Transform data based on selection type and user prompt"""
        prompt_input = Text(
            placeholder='What transformation would you like to apply?',
            layout=Layout(width='500px')
        )
        
        def on_submit(b):
            prompt = prompt_input.value
            if not prompt:
                return
                
            with self.output:
                clear_output()
                
                selection_type = selection_info['type']
                data = selection_info['data']
                
                base_prompt = self.get_prompt_for_selection(
                    selection_type,
                    data,
                    selection_info.get('headers'),
                    selection_info.get('context_rows')
                )
                
                full_prompt = {
                    **base_prompt,
                    "user_instruction": prompt
                }
                
                print("Would send to LLM:", json.dumps(full_prompt, indent=2))
                print("\nSample transformation result would appear here")
        
        submit_button = Button(description='Transform')
        submit_button.on_click(on_submit)
        
        with self.output:
            clear_output()
            display(VBox([
                prompt_input,
                submit_button
            ]))

    def handle_transform_click(self, b):
        """Handle transform button click"""
        # Force refresh of grid selection state
        self.main_grid.get_grid().refreshSelectedRows()
        
        selection_info = self.get_selection_info()
        if not selection_info:
            with self.output:
                clear_output()
                print("Please select some data first")
            return
                
        self.transform_data(selection_info)

    def handle_transform_all_click(self, b):
        """Handle transform all button click"""
        selection_info = {
            'type': 'full',
            'data': self.df.to_dict('records')
        }
        self.transform_data(selection_info)

    def display(self):
        """Display the grid and controls"""
        display(self.main_grid)
        display(HBox([self.transform_button, self.transform_all_button]))
        display(self.output)
        
# Sample data
data = {
    'title': ['Ancient Rome', 'Theory of Relativity', 'Machine Learning'],
    'content': [
        'Ancient Rome was the center of the Roman Empire that ruled across Europe from approximately 27 BC to 476 AD. Known for its architectural achievements, military prowess, and lasting influence on law and culture.',
        'Einstein\'s theory of relativity revolutionized our understanding of space, time, and gravity. It consists of two parts: special relativity (1905) and general relativity (1915).',
        'Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed. It uses statistical techniques to give computers the ability to "learn".'
    ]
}

# Create and display the grid
grid_app = SmartTransformGrid(data)
grid_app.display()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Grid(columns_fit='size_to_fit', compress_data=True, export_mode='disabled', height='300px', menu={'buttons': […

Output()